# Supplements

Notebook to make regional, national, and city-level exposure trends tables and supplement figures. <br>

Note: Pct of Global Total Comes from cpt_MS_calulations_ERA5_RH.ipynb <br>

By Cascade Tuholske Sep 2020 / Updated Feb 2021 for PNASv2 CPt

In [ ]:
#### Depdencies 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from matplotlib import rcParams
import glob

In [ ]:
#### Regressions, no intercept addition is needed because we're using SK LEARN HERE 
def lm_func(df, col):
    
    "simple linear model of a time series data, returns coef"
    
    # Get Data
    X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
    Y_stats = np.array(df.groupby('year')[col].sum()).reshape((-1, 1))

    # Add Intercept
    X_year_2 = sm.add_constant(X_year)

    # Regress
    model = sm.OLS(Y_stats, X_year_2).fit() 
        
    coef = int(model.params[1])
    #coef = int(coef)
            
    # R2 and P
    r2 = model.rsquared_adj
    p = model.pvalues[0]
    
    return coef, round(r2, 2), round(p, 3)

In [ ]:
#### Load Data
DATA = 'WBGT32_1D' # UPDATE 

# file paths
DATA_IN = "/home/cascade/projects/UrbanHeat/data/"  
FIG_OUT = "/home/cascade/projects/UrbanHeat/figures/"
FN_IN = 'processed/PNAS-DATA-v2/'+DATA+'_EXP.json'
HI_STATS = pd.read_json(DATA_IN+FN_IN, orient = 'split')

# Set scale
scale = 10**9

In [ ]:
HI_STATS.head()

In [ ]:
# Drop IDs where days 1983 = 1 with none else. throws error in regressions 

print(len(HI_STATS))
only83 = HI_STATS.groupby('ID_HDC_G0')['tot_days'].sum() == 1 # sum up total days and find those with 1 day
only83 = list(only83[only83 == True].index) # make a list of IDs
sub = HI_STATS[HI_STATS['ID_HDC_G0'].isin(only83)] # subset those IDs
bad_ids = sub[(sub['year'] == 1983) & (sub['tot_days'] == 1)] # drop those from 1983 only
drop_list = list(bad_ids['ID_HDC_G0']) # make a list
HI_STATS= HI_STATS[~HI_STATS['ID_HDC_G0'].isin(drop_list)] # drop those from the list
print(len(HI_STATS))

In [ ]:
#### Add In Meta Data (e.g. geographic data)
meta_fn = DATA_IN+'interim/GHS-UCDB-IDS.csv'
meta_data = pd.read_csv(meta_fn)

#### Merge in meta
HI_STATS = HI_STATS.merge(meta_data, on = 'ID_HDC_G0', how = 'left')

## Tables

In [ ]:
geog = 'sub-region'
loc_list = HI_STATS[geog].unique()
list(loc_list)

In [ ]:
#### Global Total People Days trend in 10^9\n - See Figure 1 or MS calculations
global_pdays = 2.217374507 * 10**9

In [ ]:
#### regional-level exposure trajectories 
fn_out = 'processed/PNAS-DATA-v2/WBGT32_1D_EXP-REGION.csv'
geog = 'sub-region'
loc_list = HI_STATS[geog].unique()

pday_coef = []
pday_p =[]
heat_coef = []
heat_p =[]
pop_coef = []
pop_p =[]
for loc in loc_list:
    print(loc)
    
    # Pdays OLS
    loc_stats = HI_STATS[HI_STATS[geog] == loc]
    coef, r2, p = lm_func(loc_stats, 'people_days')
    pday_coef.append(coef)
    pday_p.append(p)
    
    # People OLS
    loc_stats = HI_STATS[HI_STATS[geog] == loc]
    coef, r2, p = lm_func(loc_stats, 'people_days_heat')
    heat_coef.append(coef)
    heat_p.append(p)
    
    # People OLS
    loc_stats = HI_STATS[HI_STATS[geog] == loc]
    coef, r2, p = lm_func(loc_stats, 'people_days_pop')
    pop_coef.append(coef)
    pop_p.append(p)
    
df = pd.DataFrame()
df['region'] = loc_list
df['pdays_Coef'] = pday_coef
df['pdays_pvalue'] = pday_p
df['heat_Coef'] = heat_coef
df['heat_pvalue'] = heat_p
df['pop_Coef'] = pop_coef
df['pop_pvalue'] = pop_p
df['pct_globe'] = df['pdays_Coef'] / global_pdays * 100 
df.to_csv(DATA_IN+fn_out)

In [ ]:
# Check Eastern Asia
# Get Data
df = HI_STATS[HI_STATS['sub-region'] == 'Eastern Asia']
X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
Y_stats = np.array(df.groupby('year')['people_days'].sum()).reshape((-1, 1))

plt.plot(X_year, Y_stats)
X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
Y_stats = np.array(df.groupby('year')['people_days_heat'].sum()).reshape((-1, 1))
plt.plot(X_year, Y_stats, color = 'red')
plt.title('east asia')

print('pdays east asia')
print(lm_func(df, 'people_days'))

print('heat east asia')
print(lm_func(df, 'people_days_heat'))

In [ ]:
plt.plot(X_year, Y_stats)
plt.title('east asia')

In [ ]:
#### National-level exposure trajectories 
fn_out = 'processed/PNAS-DATA-v2/WBGT32_1D_EXP-COUNTRIES.csv'
geog = 'CTR_MN_NM'
loc_list = HI_STATS[geog].unique()

pday_coef = []
pday_p =[]
heat_coef = []
heat_p =[]
pop_coef = []
pop_p =[]
for loc in loc_list:
    
    print(loc)
    
    # Pdays OLS
    loc_stats = HI_STATS[HI_STATS[geog] == loc]
    coef, r2, p = lm_func(loc_stats, 'people_days')
    pday_coef.append(coef)
    pday_p.append(p)
    
    # People OLS
    loc_stats = HI_STATS[HI_STATS[geog] == loc]
    coef, r2, p = lm_func(loc_stats, 'people_days_heat')
    heat_coef.append(coef)
    heat_p.append(p)
    
    # People OLS
    loc_stats = HI_STATS[HI_STATS[geog] == loc]
    coef, r2, p = lm_func(loc_stats, 'people_days_pop')
    pop_coef.append(coef)
    pop_p.append(p)
    
df = pd.DataFrame()
df[geog] = loc_list
df['pdays_Coef'] = pday_coef
df['pdays_pvalue'] = pday_p
df['heat_Coef'] = heat_coef
df['heat_pvalue'] = heat_p
df['pop_Coef'] = pop_coef
df['pop_pvalue'] = pop_p
df['pct_globe'] = df['pdays_Coef'] / global_pdays * 100 
df.to_csv(DATA_IN+fn_out)

## Top fifty cities

In [ ]:
#### Top fifty cities
cities = pd.read_json(DATA_IN+'/processed/PNAS-DATA-v2/WBGT32_1D_TREND_EXP05.json', orient = 'split')

In [ ]:
cities = cities[['ID_HDC_G0', 'UC_NM_MN','coef_pdays']]
top50 = cities.sort_values('coef_pdays', ascending = False).head(50)
top50['pct_globe'] = top50['coef_pdays'] / global_pdays * 100 

#top50.to_csv(DATA_IN+'processed/AllDATA-GHS-ERA5-HI406-PDAYS-TOP50.csv')

In [ ]:
top50.head(50)

In [ ]:
top50.to_csv(DATA_IN+'processed/PNAS-DATA-v2/WBGT32_1D_EXP-TOP50.csv')

## Figures

In [ ]:
#### Make Supplement 2 
FN_OUT = 'FIGS7-WBGT32_1D-Final.tiff'

rcParams['mathtext.default'] = 'regular' # set text to reg, not italic 

fig, axs = plt.subplots(3, 3, figsize = (18, 16), sharex=True)
ws = 0.25
fig.subplots_adjust(wspace=ws)
scale = 10**8

#### Nigeria
country = 'Nigeria'
color = 'mediumorchid'
col = 'people_days'
df = HI_STATS[HI_STATS['CTR_MN_NM'] == country]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[0][0].plot(X_year, Y_stats, color = color, linestyle = '--')
axs[0][0].legend(labels = [country])
sns.regplot(X_year, Y_stats, color = color, ax = axs[0][0], scatter = False, truncate = True)
#axs[0][0].set_ylabel('Person-days $yr^-1$  [billions] ', fontsize = 13)

col = 'people_days_pop'
df = HI_STATS[HI_STATS['CTR_MN_NM'] == country]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[0][1].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[0][1], scatter = False, truncate = True)

col = 'people_days_heat'
df = HI_STATS[HI_STATS['CTR_MN_NM'] == country]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[0][2].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[0][2], scatter = False, truncate = True)
#axs[0][2].set_ylabel('Person-days $yr^-1$  [billions] ', fontsize = 13)

#### China
country = 'China'
color = 'olivedrab'
col = 'people_days'
df = HI_STATS[HI_STATS['CTR_MN_NM'] == country]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[1][0].plot(X_year, Y_stats, color = color, linestyle = '--')
axs[1][0].legend(labels = [country])
sns.regplot(X_year, Y_stats, color = color, ax = axs[1][0], scatter = False, truncate = True)

col = 'people_days_pop'
df = HI_STATS[HI_STATS['CTR_MN_NM'] == country]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[1][1].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[1][1], scatter = False, truncate = True)

col = 'people_days_heat'
df = HI_STATS[HI_STATS['CTR_MN_NM'] == country]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[1][2].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[1][2], scatter = False, truncate = True)

#### Italy
country = 'Italy'
color = 'tomato'
col = 'people_days'
df = HI_STATS[HI_STATS['CTR_MN_NM'] == country]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[2][0].plot(X_year, Y_stats, color = color, linestyle = '--')
axs[2][0].legend(labels = [country], loc = 'upper left')
sns.regplot(X_year, Y_stats, color = color, ax = axs[2][0], scatter = False, truncate = True)

col = 'people_days_pop'
df = HI_STATS[HI_STATS['CTR_MN_NM'] == country]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[2][1].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[2][1], scatter = False, truncate = True)

col = 'people_days_heat'
df = HI_STATS[HI_STATS['CTR_MN_NM'] == country]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[2][2].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[2][2], scatter = False, truncate = True)

# Axis labels & Titles
axs[0][0].set_title('Total Exposure', size = 15 )
axs[0][1].set_title('Trend due to population growth', size = 15)
axs[0][2].set_title('Trend due to urban warming', size = 15)
axs[2][1].set_xlabel('Year', fontsize = 15)
axs[1][0].set_ylabel('Person-days $yr^-1$  [billions] ', fontsize = 15)

# save it out
plt.savefig(FIG_OUT+FN_OUT, dpi = 300, bbox_inches='tight')

In [ ]:
#### Make Figure S3 and save it
FN_OUT = 'FIGS8-WBGT32_1D-Final.tiff'

rcParams['mathtext.default'] = 'regular' # set text to reg, not italic 

fig, axs = plt.subplots(3, 3, figsize = (18, 16), sharex=True)
ws = 0.25
fig.subplots_adjust(wspace=ws)
scale = 10**8

#### Lagos
city = 2125 
color = 'deepskyblue'
col = 'people_days'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[0][0].plot(X_year, Y_stats, color = color, linestyle = '--')
axs[0][0].legend(labels = ['Lagos'])
sns.regplot(X_year, Y_stats, color = color, ax = axs[0][0], scatter = False, truncate = True)

col = 'people_days_pop'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[0][1].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[0][1], scatter = False, truncate = True)

col = 'people_days_heat'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[0][2].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[0][2], scatter = False, truncate = True)

#### Abuja
city = 2565 #10687
color = 'darkgreen'
col = 'people_days'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[1][0].plot(X_year, Y_stats, color = color, linestyle = '--')
axs[1][0].legend(labels = ['Abuja'])
sns.regplot(X_year, Y_stats, color = color, ax = axs[1][0], scatter = False, truncate = True)

col = 'people_days_pop'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[1][1].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[1][1], scatter = False, truncate = True)

col = 'people_days_heat'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[1][2].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[1][2], scatter = False, truncate = True)

#### Kano
city = 2717
color = 'plum'
col = 'people_days'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[2][0].plot(X_year, Y_stats, color = color, linestyle = '--')
axs[2][0].legend(labels = ['Kano'])
sns.regplot(X_year, Y_stats, color = color, ax = axs[2][0], scatter = False, truncate = True)

col = 'people_days_pop'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[2][1].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[2][1], scatter = False, truncate = True)

col = 'people_days_heat'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum()/ scale)
axs[2][2].plot(X_year, Y_stats, color = color, linestyle = '--')
sns.regplot(X_year, Y_stats, color = color, ax = axs[2][2], scatter = False, truncate = True)

# Axis labels & Titles
axs[0][0].set_title('Total Exposure', size = 15 )
axs[0][1].set_title('Trend due to population growth', size = 15)
axs[0][2].set_title('Trend due to urban warming', size = 15)
axs[2][1].set_xlabel('Year', fontsize = 15)
axs[1][0].set_ylabel('Person-days $yr^-1$  [billions] ', fontsize = 15)

# save it out
plt.savefig(FIG_OUT+FN_OUT, dpi = 300, bbox_inches='tight')

## Indian Cities

In [ ]:
top50[(top50['UC_NM_MN'] == 'Delhi [IND]') | (top50['UC_NM_MN'] == 'Kolkata (Calcutta) [IND]')]

In [ ]:
#### Make Figure S3 and save it
FN_OUT = 'FIGS10-WBGT32_1D-Final.tiff'

rcParams['mathtext.default'] = 'regular' # set text to reg, not italic 

fig, axs = plt.subplots(1, 3, figsize = (16, 5), sharex=True)
ws = 0.25
fig.subplots_adjust(wspace=ws)
scale = 10**9

#### Total
city = 6955 # Delhi
color = 'cornflowerblue'
col = 'people_days'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[0].plot(X_year, Y_stats, color = color, linestyle = '-', linewidth = 3, alpha = 0.7)
axs[0].legend(labels = ['Delhi', 'Kolkata'])

city = 9691 # Kolkata 
color = 'chocolate'
col = 'people_days'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[0].plot(X_year, Y_stats, color = color, linestyle = '-', linewidth = 3, alpha = 0.7)
axs[0].legend(labels = ['Delhi', 'Kolkata'])

#### Pop
city = 6955 # Delhi
color = 'cornflowerblue'
col = 'people_days_pop'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[1].plot(X_year, Y_stats, color = color, linestyle = '-', linewidth = 3, alpha = 0.7)
axs[1].legend(labels = ['Delhi', 'v'])

city = 9691 # Kolkata 
color = 'chocolate'
col = 'people_days_pop'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[1].plot(X_year, Y_stats, color = color, linestyle = '-', linewidth = 3, alpha = 0.7)
axs[1].legend(labels = ['Delhi', 'Kolkata'])

#### Heat
city = 6955 # Delhi
color = 'cornflowerblue'
col = 'people_days_heat'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[2].plot(X_year, Y_stats, color = color, linestyle = '-', linewidth = 3, alpha = 0.7)
axs[2].legend(labels = ['Delhi', 'Kolkata'])

city = 9691 # Kolkata 
color = 'chocolate'
col = 'people_days_heat'
df = HI_STATS[HI_STATS['ID_HDC_G0'] == city]
X_year = list(df.groupby('year')['ID_HDC_G0'].mean().index)
Y_stats = list(df.groupby('year')[col].sum() / scale)
axs[2].plot(X_year, Y_stats, color = color, linestyle = '-', linewidth = 3, alpha = 0.7)
axs[2].legend(labels = ['Delhi', 'Kolkata'])

# Add text
axs[0].text(0.00, 1.1, '(A)', transform=axs[0].transAxes, size=15, fontweight='bold')
axs[1].text(1.25, 1.1, '(B)', transform=axs[0].transAxes, size=15, fontweight='bold')
axs[1].text(1.25*2, 1.1, '(C)', transform=axs[0].transAxes, size=15, fontweight='bold')
axs[0].set_title('Total Exposure', size = 15 )
axs[1].set_title('Trend due to population growth', size = 15)
axs[2].set_title('Trend due to urban warming', size = 15)
axs[1].set_xlabel('Year', fontsize = 15)
axs[0].set_ylabel('Person-days $yr^-1$  [billions] ', fontsize = 15)

# save it out
plt.savefig(FIG_OUT+FN_OUT, dpi = 300, bbox_inches='tight')

# CHIRTS vs ERA5

In [ ]:
PATH = '/home/CHIRTS/csvs/Kolkata/'
OUT = '/home/cascade/projects/UrbanHeat/figures/'
fns = sorted(glob.glob(PATH+'*'))

In [ ]:
fns

In [ ]:
#### Make Figure S3 and save it
FN_OUT = 'FIG-S2.tiff'

rcParams['mathtext.default'] = 'regular' # set text to reg, not italic 

fig, axs = plt.subplots(5, 1, figsize = (20, 14), sharex=True)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.3)

# .001
chirts = pd.read_csv('/home/CHIRTS/csvs/Kolkata/CHIRTSmax.daily.timeseries.001x001.csv')
era5 = pd.read_csv('/home/CHIRTS/csvs/Kolkata/ERA5_Tmax.daily.timeseries.001x001.csv')
axs[0].plot(chirts['year'],chirts['151'])
axs[0].plot(era5['year'],era5['151'])
axs[0].set_title('Mean Tmax 1 x 1 pixel', fontsize = 15)
axs[0].legend(labels = ['CHIRTS-daily', 'ERA5'], fontsize = 15, loc = 'upper right')
axs[0].set_xlim([1982,2016]);
#axs[0].set_xlabel('Year', fontsize = 15)
axs[0].set_ylabel('Tmax °C', fontsize = 15)

# .005
chirts = pd.read_csv('/home/CHIRTS/csvs/Kolkata/CHIRTSmax.daily.timeseries.005x005.csv')
era5 = pd.read_csv('/home/CHIRTS/csvs/Kolkata/ERA5_Tmax.daily.timeseries.005x005.csv')
axs[1].plot(chirts['year'],chirts['151'])
axs[1].plot(era5['year'],era5['151'])
axs[1].set_title('Mean Tmax 5 x 5 pixels', fontsize = 15)
axs[1].legend(labels = ['CHIRTS-daily', 'ERA5'], fontsize = 15, loc = 'upper right')
axs[1].set_xlim([1982,2016]);
#axs[1].set_xlabel('Year', fontsize = 15)
axs[1].set_ylabel('Tmax °C', fontsize = 15)

# 0.011
chirts = pd.read_csv('/home/CHIRTS/csvs/Kolkata/CHIRTSmax.daily.timeseries.011x011.csv')
era5 = pd.read_csv('/home/CHIRTS/csvs/Kolkata/ERA5_Tmax.daily.timeseries.011x011.csv')
axs[2].plot(chirts['year'],chirts['151'])
axs[2].plot(era5['year'],era5['151'])
axs[2].set_title('Mean Tmax 11 x 11 pixels', fontsize = 15)
axs[2].legend(labels = ['CHIRTS-daily', 'ERA5'], fontsize = 15, loc = 'upper right')
axs[2].set_xlim([1982,2016]);
#axs[2].set_xlabel('Year', fontsize = 15)
axs[2].set_ylabel('Tmax °C', fontsize = 15)

# 025
chirts = pd.read_csv('/home/CHIRTS/csvs/Kolkata/CHIRTSmax.daily.timeseries.025x025.csv')
era5 = pd.read_csv('/home/CHIRTS/csvs/Kolkata/ERA5_Tmax.daily.timeseries.025x025.csv')
axs[3].plot(chirts['year'],chirts['151'])
axs[3].plot(era5['year'],era5['151'])
axs[3].set_title('Mean Tmax 25 x 25 pixels', fontsize = 15)
axs[3].legend(labels = ['CHIRTS-daily', 'ERA5'], fontsize = 15, loc = 'upper right')
axs[3].set_xlim([1982,2016]);
#axs[3].set_xlabel('Year', fontsize = 15)
axs[3].set_ylabel('Tmax °C', fontsize = 15)

# 051
chirts = pd.read_csv('/home/CHIRTS/csvs/Kolkata/CHIRTSmax.daily.timeseries.051x051.csv')
era5 = pd.read_csv('/home/CHIRTS/csvs/Kolkata/ERA5_Tmax.daily.timeseries.051x051.csv')
axs[4].plot(chirts['year'],chirts['151'])
axs[4].plot(era5['year'],era5['151'])
axs[4].set_title('Mean Tmax 51 x 51 pixels', fontsize = 15)
axs[4].legend(labels = ['CHIRTS-daily', 'ERA5'], fontsize = 15, loc = 'upper right')
axs[4].set_xlim([1982,2016]);
axs[4].set_xlabel('Year', fontsize = 15)
axs[4].set_ylabel('Tmax °C', fontsize = 15)

# Add text
# axs[0].text(0.00, 1.05, '(A)', transform=axs[0].transAxes, size=15, fontweight='bold')
# axs[0].text(0.00, -.25, '(B)', transform=axs[0].transAxes, size=15, fontweight='bold')
# axs[0].text(0.00, -1.55, '(C)', transform=axs[0].transAxes, size=15, fontweight='bold')
# axs[0].text(0.00, -1.75, '(D)', transform=axs[0].transAxes, size=15, fontweight='bold')
# axs[0].text(0.00, -2.55, '(F)', transform=axs[0].transAxes, size=15, fontweight='bold')

plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

# save it out
plt.savefig(OUT+FN_OUT, dpi = 300, bbox_inches='tight')